In [ ]:
# load the pridict, deepprime and conventional ML models
import os
import sys
import numpy as np
import pandas as pd
import pickle
import torch
import skorch

from models.deepprime import predict_deep_prime
from models.pridict import predict_pridict

dataf = 'pd-hek293t-pe2.csv'

data_dp = f'dp-{dataf}'
data_pd = f'pd-{dataf}'

predict_pridict(data_pd, num_features=24, device='cuda' if torch.cuda.is_available() else 'cpu', dropout=0)